## Run the next two cells to load the log from one of today's runs.

In [48]:
import qgrid
import pandas as pd
import time
import warnings
from measurement_directory import *
from numpy import nan

def translate_stringy_list(stringy_list):
    #pd.read_csv imports listboundvariables dtype as a string, we want it in list form
    final_list = []
    stringy_list = stringy_list.split(',')
    for var in stringy_list:
        final_list.append(var.translate({ord(c): None for c in '[ ]\,\''}))
    return final_list

def save_image_log(event, qgrid_widget):
    df_display = qgrid_widget.get_changed_df()
    qgrid_widget.close()
    df = pd.read_csv(filename)
    for idx in range(len(df_display)):
        for column in df_display.columns:
            df.loc[idx, column] = df_display.loc[idx, column]
    df.to_csv(filename, index = False)
    qgrid_widget = edit_image_log()
    qgrid_widget.on('cell_edited',save_image_log)
    display(qgrid_widget)

def edit_image_log(new_column = False, extra_display_vars = []):
    #new_columns accepts a dictionary of {column_name: default value}, e.g. {'filter_in': False, 'OD':4}
    #set default value of string dtypes as '[space]' for easier parsing
    
#     try: #save the edited log if it was opened and edited
#         save_image_log(filename)
#     except:
#         pass
    df = pd.read_csv(filename, dtype={'badshot': 'bool'})
    display_vars = []
    
    if 'editable_variables' in df.columns:
        editable_vars = translate_stringy_list(df['editable_variables'][0])
    else:
        editable_vars = []
    
    if new_column:
        column_name = input('enter new column name (use CamelCase): ')
        if column_name not in df.columns:
            df[column_name] = nan
            editable_vars += [column_name]
            df['editable_variables'] = str(editable_vars)
        else:
            warnings.warn('column \'' + column_name + '\' already exists')
    
    list_bound_vars = translate_stringy_list(df['ListBoundVariables'][0])
    if 'editable_variables' in df.columns:
        editable_vars = translate_stringy_list(df['editable_variables'][0])
    else:
        editable_vars = []
        
    if 'analyzed_variables' in df.columns:
        analyzed_vars = translate_stringy_list(df['analyzed_variables'][0])
        #parse analyzed_vars
        for i in range(len(analyzed_vars)):
            if analyzed_vars[i] not in df.columns:
                for column in df.columns:
                    if analyzed_vars[i] == column.translate({ord(c): None for c in '[ ]\,\''}):
                        analyzed_vars[i] = column
    else:
        analyzed_vars = []
        
    display_vars += ['badshot', 'notes'] + editable_vars + list_bound_vars  + extra_display_vars + analyzed_vars + ['runtime']
    if 'editable_variables' in df.columns:
        display_vars += ['editable_variables']
    
    df_display = df[display_vars]
    qgrid_widget = qgrid.show_grid(df_display, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})
    return qgrid_widget

# def edit_image_log(new_columns = {}):
#     #new_columns accepts a dictionary of {column_name: default value}, e.g. {'filter_in': False, 'OD':4}
#     #set default value of string dtypes as '[space]' for easier parsing
    
# #     try: #save the edited log if it was opened and edited
# #         save_image_log(filename)
# #     except:
# #         pass
#     df = pd.read_csv(filename, dtype={'badshot': 'bool'})
# #     for column_name in new_columns:
# #         if column_name not in df.columns:
# #             df[column_name] = new_columns[column_name]
# #         else:
# #             warnings.warn('column \'' + column_name + '\' already exists')
    
#     list_bound_vars = translate_stringy_list(df['ListBoundVariables'][0])
# #     if 'editable_variables' in df.columns:
# #         editable_vars = translate_stringy_list(df['editable_variables'][0])
# #         editable_vars = editable_vars + [column_name for column_name in new_columns]
# #     else:
# #         editable_vars = [column_name for column_name in new_columns]
# #     if len(editable_vars) > 0:
# #         df['editable_variables'] = str(editable_vars)
    
# #     display_vars = ['badshot', 'notes'] + list_bound_vars + ['fake analysis'] + editable_vars + ['runtime']
#     display_vars = ['badshot', 'notes'] + list_bound_vars + ['runtime', 'analyzed_variables']
#     df_display = df[display_vars]
#     qgrid_widget = qgrid.show_grid(df_display, grid_options = {'fullWidthRows' : False})
#     return qgrid_widget

import ipywidgets as widgets
from ipywidgets import interact

def add_new_column(new_column_in = False):
    global new_column
    new_column = new_column_in
    
mode_select_widget = interact(add_new_column,new_column_in=[False, True])

interactive(children=(Dropdown(description='new_column_in', options=(False, True), value=False), Output()), _d…

In [46]:
global filename
filename = measurement_directory() + r'\image_log.csv'
qgrid_widget = edit_image_log(new_column)
qgrid_widget.on('cell_edited',save_image_log)
display(qgrid_widget)

Enter name for this set of runs:foo
enter new column name (use CamelCase): blah


C:\Users\Alex\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: UserWarning: column 'blah' already exists


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

['foo']